# ****FLUSHING ANALYTICS****


In [2]:
!pip install pandas
!pip install pyproj
!pip install scikit-learn

StatementMeta(, , , Waiting, , Waiting)

     ━━━━━━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━ 5.2/8.3 MB 155.3 MB/s eta 0:00:01

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 124.4 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse


# file_path = '/EllaTesting/Files/Repairs_2018_2023.xlsx'

# print(file_path)

all_hydrants_filepath = "/lakehouse/default/Files/Flushing/Qry_AllHY.xlsx"
flushing_history_filepath = "/lakehouse/default/Files/Flushing/Qry_FlushHistory.xlsx"

all_hydrants_df = pd.read_excel(all_hydrants_filepath)
flushing_history_df = pd.read_excel(flushing_history_filepath)

StatementMeta(, , , Waiting, , Waiting)

In [4]:
import copy

copied_all_hydrants_df = copy.deepcopy(all_hydrants_df)

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
copied_all_hydrants_df = copied_all_hydrants_df[copied_all_hydrants_df['FLSHFRE'].notnull()]
copied_all_hydrants_df.describe()

StatementMeta(, , , Waiting, , Waiting)

,ADDRKEY,AUXVALVE,BARRELSIZE,IMSRO_COMPHY_COMPKEY,COMPLEXKEY,FEEDERDIAM,FEEDERLEN,GISSTATIC,HT,INTKEY,...,EMASSTCRTL,EMASSTCRDT,EMASSTRISK,EMASTRSKDT,EMDISTRICT,EMEQPMNTCL,EMEQPMNTTP,EMASSTDESC,EMCOND,EMCONDDATE
count,2389.000000,0.0,2389.0,2.389000e+03,2389.0,2389.000000,2389.000000,0.0,2389.0,2389.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,81076.322729,NaN,0.0,3.362129e+05,1.0,63.729594,4.024884,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1003.000000,NaN,0.0,1.002000e+03,1.0,0.000000,0.000000,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,3243.000000,NaN,0.0,9.067000e+03,1.0,0.000000,1.200000,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,5478.000000,NaN,0.0,1.939500e+04,1.0,0.000000,2.291000,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,129011.000000,NaN,0.0,7.158620e+05,1.0,150.000000,3.500000,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,437401.000000,NaN,0.0,1.312755e+06,1.0,150.000000,1166.540000,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,122051.138142,NaN,0.0,4.165480e+05,0.0,74.163875,29.154112,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
copied_flushing_history_df = copy.deepcopy(flushing_history_df)

StatementMeta(, , , Waiting, , Waiting)

In [6]:
#Item 1: Show the description of each column in the df

num_columns = copied_all_hydrants_df.shape[1]
window_size = 20

for i in range(0, num_columns, window_size):
    subset = copied_all_hydrants_df.iloc[:, i:i+window_size]
    description = subset.describe(include='all')
    print(description)

StatementMeta(, , , Waiting, , Waiting)

              ADDRKEY ADDRQUAL  AREA       ASBLT  AUXVALVE  BARRELSIZE COLOR  \
count     2389.000000     1580  2387        1237       0.0      2389.0  1989   
unique            NaN     1248   132        1148       NaN         NaN     3   
top               NaN        4   B31  SEECOMMENT       NaN         NaN  BLUE   
freq              NaN       23   145          42       NaN         NaN  1596   
mean     81076.322729      NaN   NaN         NaN       NaN         0.0   NaN   
std     122051.138142      NaN   NaN         NaN       NaN         0.0   NaN   
min       1003.000000      NaN   NaN         NaN       NaN         0.0   NaN   
25%       3243.000000      NaN   NaN         NaN       NaN         0.0   NaN   
50%       5478.000000      NaN   NaN         NaN       NaN         0.0   NaN   
75%     129011.000000      NaN   NaN         NaN       NaN         0.0   NaN   
max     437401.000000      NaN   NaN         NaN       NaN         0.0   NaN   

        IMSRO_COMPHY_COMPKEY  COMPLEXKE

In [7]:
#Item 1: Count the number of null values in each column

for i in range(0, num_columns, window_size):
    subset = copied_all_hydrants_df.iloc[:, i:i+window_size]
    description = subset.isna().sum()
    print(description)

StatementMeta(, , , Waiting, , Waiting)

ADDRKEY                    0
ADDRQUAL                 809
AREA                       2
ASBLT                   1152
AUXVALVE                2389
BARRELSIZE                 0
COLOR                    400
IMSRO_COMPHY_COMPKEY       0
COMPLEXKEY                 0
DISTRICT                   0
FEEDERDIAM                 0
FEEDERLEN                  0
FEEDERTYPE              1380
GISSTATIC               2389
HT                         0
INSTDATE                1185
INTKEY                     0
LOC                        0
MAINKEY                    0
MAPNO                   1867
dtype: int64
MFGKEY          0
MODELNO       690
OBST         2389
OUTLSZ1         0
OUTLSZ2         0
OUTLSZ3         0
OUTLSZ4         0
OWN             0
PACKING      2389
PAINTTYPE    2389
PRCLKEY         0
PRESZONE        3
SEGKEY          0
SERNO        2388
SERVSTAT        0
SLKEY           0
SPECINST     1517
STKEY           0
SUBAREA         2
UNITID          0
dtype: int64
UNITTYPE                  0
WVKEY 

In [8]:
# Item 3: Clean data, replace nulls with values in some columns
copied_all_hydrants_df['UNITTYPE'] = copied_all_hydrants_df['UNITTYPE'].fillna('STDHY')

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
copied_all_hydrants_df['UNITTYPE'].describe()

StatementMeta(, , , Waiting, , Waiting)

count       2389
unique         2
top       HYBLWF
freq        2350
Name: UNITTYPE, dtype: object

In [9]:
# Item 9: Coordinates
copied_all_hydrants_df['XCOORD'].head()
copied_all_hydrants_df['XCOORD_float'] = pd.to_numeric(copied_all_hydrants_df['XCOORD'], errors='coerce')
copied_all_hydrants_df['YCOORD_float'] = pd.to_numeric(copied_all_hydrants_df['YCOORD'], errors='coerce')
copied_all_hydrants_df['XCOORD_float'].head()
copied_all_hydrants_df.describe()

StatementMeta(, , , Waiting, , Waiting)

,ADDRKEY,AUXVALVE,BARRELSIZE,IMSRO_COMPHY_COMPKEY,COMPLEXKEY,FEEDERDIAM,FEEDERLEN,GISSTATIC,HT,INTKEY,...,EMEQPMNTCL,EMEQPMNTTP,EMASSTDESC,EMCOND,EMCONDDATE,XCOORD_float,YCOORD_float,Latitude,Longitude,Geo_Cluster
count,2389.000000,0.0,2389.0,2.389000e+03,2389.0,2389.000000,2389.000000,0.0,2389.0,2389.0,...,0.0,0.0,0.0,0.0,0.0,2389.000000,2.389000e+03,2389.000000,2389.000000,2389.0
mean,81076.322729,NaN,0.0,3.362129e+05,1.0,63.729594,4.024884,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,603585.008429,4.835625e+06,43.666383,-121.715269,0.0
min,1003.000000,NaN,0.0,1.002000e+03,1.0,0.000000,0.000000,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,574107.000000,4.817268e+06,43.500369,-122.077736,0.0
25%,3243.000000,NaN,0.0,9.067000e+03,1.0,0.000000,1.200000,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,599879.840000,4.825955e+06,43.578955,-121.760724,0.0
50%,5478.000000,NaN,0.0,1.939500e+04,1.0,0.000000,2.291000,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,603124.950000,4.833903e+06,43.651207,-121.719748,0.0
75%,129011.000000,NaN,0.0,7.158620e+05,1.0,150.000000,3.500000,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,608432.340000,4.842763e+06,43.731067,-121.655763,0.0
max,437401.000000,NaN,0.0,1.312755e+06,1.0,150.000000,1166.540000,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,616887.720000,4.869890e+06,43.976043,-121.552075,0.0
std,122051.138142,NaN,0.0,4.165480e+05,0.0,74.163875,29.154112,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,6786.099251,1.158453e+04,0.104866,0.082829,0.0


In [10]:
def utm_to_latlon(df, xccord, ycoord, utm_zone):
    utm_proj = Proj(proj='utm', zone=utm_zone, ellps='WGS84')
    lonlat = utm_proj(df[xccord].values, df[ycoord].values, inverse=True)
    df['Latitude'] = lonlat[1]
    df['Longitude'] = lonlat[0]
    return df

StatementMeta(, , , Waiting, , Waiting)

In [11]:
from pyproj import Proj, transform
copied_all_hydrants_df = utm_to_latlon(copied_all_hydrants_df, 'XCOORD_float', 'YCOORD_float', utm_zone=10)
copied_all_hydrants_df.head()
copied_all_hydrants_df.describe()

StatementMeta(, , , Waiting, , Waiting)

,ADDRKEY,AUXVALVE,BARRELSIZE,IMSRO_COMPHY_COMPKEY,COMPLEXKEY,FEEDERDIAM,FEEDERLEN,GISSTATIC,HT,INTKEY,...,EMEQPMNTCL,EMEQPMNTTP,EMASSTDESC,EMCOND,EMCONDDATE,XCOORD_float,YCOORD_float,Latitude,Longitude,Geo_Cluster
count,2389.000000,0.0,2389.0,2.389000e+03,2389.0,2389.000000,2389.000000,0.0,2389.0,2389.0,...,0.0,0.0,0.0,0.0,0.0,2389.000000,2.389000e+03,2389.000000,2389.000000,2389.0
mean,81076.322729,NaN,0.0,3.362129e+05,1.0,63.729594,4.024884,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,603585.008429,4.835625e+06,43.666383,-121.715269,0.0
min,1003.000000,NaN,0.0,1.002000e+03,1.0,0.000000,0.000000,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,574107.000000,4.817268e+06,43.500369,-122.077736,0.0
25%,3243.000000,NaN,0.0,9.067000e+03,1.0,0.000000,1.200000,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,599879.840000,4.825955e+06,43.578955,-121.760724,0.0
50%,5478.000000,NaN,0.0,1.939500e+04,1.0,0.000000,2.291000,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,603124.950000,4.833903e+06,43.651207,-121.719748,0.0
75%,129011.000000,NaN,0.0,7.158620e+05,1.0,150.000000,3.500000,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,608432.340000,4.842763e+06,43.731067,-121.655763,0.0
max,437401.000000,NaN,0.0,1.312755e+06,1.0,150.000000,1166.540000,NaN,0.0,1.0,...,NaN,NaN,NaN,NaN,NaN,616887.720000,4.869890e+06,43.976043,-121.552075,0.0
std,122051.138142,NaN,0.0,4.165480e+05,0.0,74.163875,29.154112,NaN,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,6786.099251,1.158453e+04,0.104866,0.082829,0.0


In [12]:
from sklearn.cluster import DBSCAN

dbscan = DBSCAN(eps=0.1, min_samples=5)
copied_all_hydrants_df = copied_all_hydrants_df.dropna(subset=['Longitude', 'Latitude'])
copied_all_hydrants_df['Latitude'] = copied_all_hydrants_df['Latitude'].replace([np.inf, -np.inf], np.nan).dropna()
copied_all_hydrants_df['Longitude'] = copied_all_hydrants_df['Longitude'].replace([np.inf, -np.inf], np.nan).dropna()
# print(copied_all_hydrants_df.sort_values(by=['Latitude'], ascending=False).head())
copied_all_hydrants_df['Latitude'] = copied_all_hydrants_df['Latitude'].dropna()
copied_all_hydrants_df['Longitude'] = copied_all_hydrants_df['Longitude'].dropna()
copied_all_hydrants_df.dropna(subset=['Longitude', 'Latitude'], inplace=True)
print(copied_all_hydrants_df['Latitude'].describe())
print(copied_all_hydrants_df['Longitude'].describe())
print(copied_all_hydrants_df['Latitude'].isna().sum())
print(copied_all_hydrants_df['Longitude'].isna().sum())

StatementMeta(, , , Waiting, , Waiting)

count    2389.000000
mean       43.666383
std         0.104866
min        43.500369
25%        43.578955
50%        43.651207
75%        43.731067
max        43.976043
Name: Latitude, dtype: float64
count    2389.000000
mean     -121.715269
std         0.082829
min      -122.077736
25%      -121.760724
50%      -121.719748
75%      -121.655763
max      -121.552075
Name: Longitude, dtype: float64
0
0


In [13]:
copied_all_hydrants_df['Geo_Cluster'] = dbscan.fit_predict(copied_all_hydrants_df[['Longitude', 'Latitude']])

StatementMeta(, , , Waiting, , Waiting)

2024-07-11:14:19:06,828 ERROR    [synapse_mlflow_utils.py:420] [fabric mlflow plugin]: <class 'synapse.ml.mlflow.tracking_store.TridentMLflowTrackingStore'>.create_experiment exception  create experiment get return code: 404, body: b'{"error":{"code":"PowerBIFeatureDisabled","pbi.error":{"code":"PowerBIFeatureDisabled","parameters":{},"details":[],"exceptionCulprit":1}}}', headers: {'Cache-Control': 'no-store, must-revalidate, no-cache', 'Pragma': 'no-cache', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json; charset=utf-8', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'RequestId': 'cb652ef4-a94c-4715-9b7a-51bcb642664d', 'Access-Control-Expose-Headers': 'RequestId', 'Date': 'Thu, 11 Jul 2024 14:19:06 GMT'}
2024-07-11:14:19:06,830 ERROR    [default_experiment_registry.py:43] Error find or created notebook/SJD default experiment, please call mlflow.set_experiment to set a valid one befo

2024-07-11:14:19:06,831 ERROR    [synapse_mlflow_utils.py:420] [fabric mlflow plugin]: <class 'synapse.ml.mlflow.default_experiment_registry.SynapseNotebookDefaultExperimentProvider'>.get_experiment_id exception  create experiment get return code: 404, body: b'{"error":{"code":"PowerBIFeatureDisabled","pbi.error":{"code":"PowerBIFeatureDisabled","parameters":{},"details":[],"exceptionCulprit":1}}}', headers: {'Cache-Control': 'no-store, must-revalidate, no-cache', 'Pragma': 'no-cache', 'Transfer-Encoding': 'chunked', 'Content-Type': 'application/json; charset=utf-8', 'Strict-Transport-Security': 'max-age=31536000; includeSubDomains', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'RequestId': 'cb652ef4-a94c-4715-9b7a-51bcb642664d', 'Access-Control-Expose-Headers': 'RequestId', 'Date': 'Thu, 11 Jul 2024 14:19:06 GMT'}
2024-07-11:14:19:06,891 ERROR    [synapse_mlflow_utils.py:413] [fabric mlflow plugin]: <class 'mlflow.store.tracking.rest_store.RestStore'>._call_endpoint

2024-07-11:14:19:06,893 ERROR    [synapse_mlflow_utils.py:413] [fabric mlflow plugin]: <class 'mlflow.store.tracking.rest_store.RestStore'>.create_run exception BAD_REQUEST: Response: {'Message': 'Experiment Id should not be empty and needs to be a valid GUID.', 'Source': 'ML', 'error_code': 'BAD_REQUEST'}


In [14]:
copied_all_hydrants_df.head()

StatementMeta(, , , Waiting, , Waiting)

,ADDRKEY,ADDRQUAL,AREA,ASBLT,AUXVALVE,BARRELSIZE,COLOR,IMSRO_COMPHY_COMPKEY,COMPLEXKEY,DISTRICT,...,EMEQPMNTCL,EMEQPMNTTP,EMASSTDESC,EMCOND,EMCONDDATE,XCOORD_float,YCOORD_float,Latitude,Longitude,Geo_Cluster
65,1494,20 - 22,B31,NaN,NaN,0,BLUE,1673,1,C,...,NaN,NaN,NaN,NaN,NaN,597828.69,4841977.83,43.724380,-121.785410,0
76,1509,16,B31,NaN,NaN,0,BLUE,1696,1,C,...,NaN,NaN,NaN,NaN,NaN,596099.18,4844170.61,43.744345,-121.806486,0
126,2226,56,B31,NaN,NaN,0,BLUE,2847,1,C,...,NaN,NaN,NaN,NaN,NaN,596309.85,4841454.28,43.719866,-121.804357,0
138,211559,NaN,B06,NaN,NaN,0,ORANGE,2863,1,C,...,NaN,NaN,NaN,NaN,NaN,602040.45,4833238.58,43.645143,-121.734786,0
139,2246,7,B12,NaN,NaN,0,BLUE,2864,1,C,...,NaN,NaN,NaN,NaN,NaN,603629.16,4841098.82,43.715680,-121.713581,0


In [15]:
copied_all_hydrants_df['Geo_Cluster'].describe()

StatementMeta(, , , Waiting, , Waiting)

count    2389.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: Geo_Cluster, dtype: float64

In [ ]:
print(copied_all_hydrants_df.shape)
na_copied_all_hydrants_df = copy.deepcopy(copied_all_hydrants_df)
na_copied_all_hydrants_df.shape

StatementMeta(, , , Waiting, , Waiting)

(2389, 123)


(2389, 123)

In [ ]:
na_copied_all_hydrants_df = na_copied_all_hydrants_df.fillna('Unknown')
na_copied_all_hydrants_df

StatementMeta(, , , Waiting, , Waiting)

,ADDRKEY,ADDRQUAL,AREA,ASBLT,AUXVALVE,BARRELSIZE,COLOR,IMSRO_COMPHY_COMPKEY,COMPLEXKEY,DISTRICT,...,EMEQPMNTCL,EMEQPMNTTP,EMASSTDESC,EMCOND,EMCONDDATE,XCOORD_float,YCOORD_float,Latitude,Longitude,Geo_Cluster
65,1494,20 - 22,B31,Unknown,Unknown,0,BLUE,1673,1,C,...,Unknown,Unknown,Unknown,Unknown,Unknown,597828.69,4841977.83,43.724380,-121.785410,0
76,1509,16,B31,Unknown,Unknown,0,BLUE,1696,1,C,...,Unknown,Unknown,Unknown,Unknown,Unknown,596099.18,4844170.61,43.744345,-121.806486,0
126,2226,56,B31,Unknown,Unknown,0,BLUE,2847,1,C,...,Unknown,Unknown,Unknown,Unknown,Unknown,596309.85,4841454.28,43.719866,-121.804357,0
138,211559,Unknown,B06,Unknown,Unknown,0,ORANGE,2863,1,C,...,Unknown,Unknown,Unknown,Unknown,Unknown,602040.45,4833238.58,43.645143,-121.734786,0
139,2246,7,B12,Unknown,Unknown,0,BLUE,2864,1,C,...,Unknown,Unknown,Unknown,Unknown,Unknown,603629.16,4841098.82,43.715680,-121.713581,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40336,434890,HYDRANT UPDATED BASED ON SERVICE REQUEST 49555...,C36,77909-D_IF,Unknown,0,Unknown,1307858,1,V,...,Unknown,Unknown,Unknown,Unknown,Unknown,591878.10,4857846.60,43.867992,-121.856553,0
40502,436251,OPP LOT 89 - UPDATED TO A FLUSHING HYDRANT AS ...,B34,79354-D_IF,Unknown,0,Unknown,1285373,1,C,...,Unknown,Unknown,Unknown,Unknown,Unknown,594001.50,4835271.80,43.664506,-121.834087,0
40503,436251,OPP 49 - UPDATED TO A FLUSHING HYDRANT AS REQU...,B34,79351-D_IF,Unknown,0,Unknown,1312734,1,C,...,Unknown,Unknown,Unknown,Unknown,Unknown,593818.30,4835393.10,43.665622,-121.836337,0
40504,436247,AT BLOCK 133 - UPDATED TO A FLUSHING HYDRANT A...,B34,79340-D_IF,Unknown,0,Unknown,1312743,1,C,...,Unknown,Unknown,Unknown,Unknown,Unknown,594066.60,4835547.30,43.666978,-121.833231,0


In [ ]:
knn_copied_all_hydrants_df = copy.deepcopy(copied_all_hydrants_df)

StatementMeta(, , , Waiting, , Waiting)

In [ ]:
copied_all_hydrants_df['FLSHFRE'].describe()


StatementMeta(, e28b2cac-e366-4710-bce2-fbaf83bf91ac, 86, Finished, Available, Finished)

count       2389
unique        12
top       1 YEAR
freq        1221
Name: FLSHFRE, dtype: object

In [ ]:
copied_all_hydrants_df['FLSHFRE'].unique()

StatementMeta(, , , Waiting, , Waiting)

array(['1 YEAR', '3 MONTHS', '6 MONTHS', '3 DAYS', '1 WEEK', '4 MONTHS',
       '2 WEEKS', '1 MONTH', '2 MONTHS', '3 WEEKS', '2 DAYS', 'DAILY'],
      dtype=object)